# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

> 2003 has fewer columns than the other files, so exluding 2003 allows us to keep more data.

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [1]:
from project_data_Miertschin import common_columns, ll_dist_dict, ll_code_dict, code_name_dict, ll_idnamedist_dict

In [2]:
import pandas as pd
from dfply import *
from glob import glob
import re
from toolz import first
from more_dfply import recode

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [3]:
files = glob('../MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
files

['../MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 '../MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

In [4]:
FILE_NAME_RE = re.compile(r'^\.\./MinneMUDAC_raw_files/(20\d\d_metro_tax_parcels)\.txt$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 
file_names = lambda files: [file_name(p) for p in files]
years= file_names(files)
years

['2004_metro_tax_parcels',
 '2005_metro_tax_parcels',
 '2006_metro_tax_parcels',
 '2007_metro_tax_parcels',
 '2008_metro_tax_parcels',
 '2009_metro_tax_parcels',
 '2010_metro_tax_parcels',
 '2011_metro_tax_parcels',
 '2012_metro_tax_parcels',
 '2013_metro_tax_parcels',
 '2014_metro_tax_parcels']

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [76]:
parcel_files = [pd.read_csv(file,chunksize=500,sep='|',dtype = {'centroid_lat': str,'centroid_long': str, 'PIN': str, 'Year':str}) for file in files]

In [77]:
first_chunks = [first(df) for df in parcel_files]

In [78]:
first_chunk = first_chunks[0]
first_chunk2 = first_chunk.head()
first_chunk2

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739
1,0.0,0.93,NaN,NaN,N,NaN,24457.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2004,55005.0,NaN,45.41354,-93.2701
2,0.0,8.75,NaN,NaN,N,NaN,24442.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2004,55005.0,NaN,45.41318,-93.27344
3,0.0,11.17,NaN,NaN,N,NaN,410.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2004,55005.0,NaN,45.41167,-93.27684
4,0.0,14.46,NaN,NaN,N,NaN,480.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2004,55070.0,NaN,45.41169,-93.27849


In [20]:
new_chunk = (first_chunk2 
             >> select(common_columns)
             >> mutate(
                 lat_long = pd.Series(zip(first_chunk2.centroid_lat,first_chunk2.centroid_long)))
             >> mutate(
                 lake_code = recode(X.lat_long,ll_code_dict),
                 distance_to_lake = recode(X.lat_long,ll_dist_dict))
             >> mutate(
                 lake_name = recode(X.lake_code,code_name_dict))
             >> filter_by(X.distance_to_lake <= 1600)
            )

In [21]:
new_chunk

,GARAGE,XUSE1_DESC,GREEN_ACRE,Year,COUNTY_ID,OWNER_MORE,OWN_ADD_L1,OWN_ADD_L2,SUFFIX_DIR,USE1_DESC,...,FIN_SQ_FT,OPEN_SPACE,STREETNAME,EMV_TOTAL,USE3_DESC,XUSE4_DESC,lat_long,lake_code,distance_to_lake,lake_name
0,NaN,NaN,N,2004,3,NaN,24457 DOGWOOD ST NW,BETHEL,NaN,NaN,...,0.0,N,NaN,23964.0,NaN,NaN,"(45.41332, -93.26739)",02007501-01,942.795032,Moore Lake
1,NaN,NaN,N,2004,3,NaN,24457 DOGWOOD ST NW,BETHEL,NW,NaN,...,0.0,N,DOGWOOD,171837.0,NaN,NaN,"(45.41354, -93.2701)",27008902-01,348.834975,Shady Oak Lake
2,NaN,NaN,N,2004,3,NaN,24442 DOGWOOD ST NW,ST FRANCIS,NW,NaN,...,0.0,N,DOGWOOD,195751.0,NaN,NaN,"(45.41318, -93.27344)",10001300-01,1055.720786,Susan Lake
4,NaN,NaN,N,2004,3,NaN,480 245TH AVE NW,EAST BETHEL,NW,NaN,...,0.0,N,245TH,204359.0,NaN,NaN,"(45.41169, -93.27849)",27011601-01,413.594909,Rice Elm Lake


In [60]:
new_chunk.Year

0    2004
1    2004
2    2004
4    2004
Name: Year, dtype: int64

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

In [22]:
def add_lake_columns(df):
    new_df = (df 
                 >> select(common_columns)
                 >> mutate(
                     lat_long = pd.Series(zip(df.centroid_lat,df.centroid_long)))
                 >> mutate(
                     lake_code = recode(X.lat_long,ll_code_dict),
                     distance_to_lake = recode(X.lat_long,ll_dist_dict))
                 >> mutate(
                     lake_name = recode(X.lake_code,code_name_dict))
                 >> filter_by(X.distance_to_lake <= 1600)
             )
    return new_df

In [24]:
new_first_chunks = [add_lake_columns(chunk) for chunk in first_chunks]

In [25]:
[chunk.head() for chunk in new_first_chunks]

[   GARAGE  XUSE1_DESC GREEN_ACRE  Year  COUNTY_ID  OWNER_MORE  \
 0     NaN         NaN          N  2004          3         NaN   
 1     NaN         NaN          N  2004          3         NaN   
 2     NaN         NaN          N  2004          3         NaN   
 4     NaN         NaN          N  2004          3         NaN   
 5     NaN         NaN          N  2004          3         NaN   
 
             OWN_ADD_L1   OWN_ADD_L2 SUFFIX_DIR  USE1_DESC  ... FIN_SQ_FT  \
 0  24457 DOGWOOD ST NW       BETHEL        NaN        NaN  ...       0.0   
 1  24457 DOGWOOD ST NW       BETHEL         NW        NaN  ...       0.0   
 2  24442 DOGWOOD ST NW   ST FRANCIS         NW        NaN  ...       0.0   
 4     480 245TH AVE NW  EAST BETHEL         NW        NaN  ...       0.0   
 5     500 LAFAYETTE RD      ST PAUL        NaN        NaN  ...       0.0   
 
    OPEN_SPACE STREETNAME  EMV_TOTAL  USE3_DESC XUSE4_DESC  \
 0           N        NaN    23964.0        NaN        NaN   
 1           N

6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PIN`.  Explain why this should make a proper primary key for the combined data. <br>

In [54]:
pin_set=set()
row_num=0

for file in first_chunks:
    pin_set = pin_set.union(set(file.PIN))
    row_num=row_num+len(file)    

print(row_num)
print(len(pin_set))

5500
2136


Even when iterating over just the first chunks, there are more rows than unique PINs. Therefore, PIN cannot be used as an ID.

The new column adding year to PIN should make the keys unique by adding more info to the key.

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PIN`) to a given chunk.

In [82]:
def add_key(df):
    df2 = (df
           >> mutate(key = X.Year.str.cat(X.PIN, sep='-'))
          )
    return df2

In [83]:
add_key(first_chunk2)

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,key
0,0.0,8.03,NaN,NaN,N,NaN,NaN,NaN,SAINT FRANCIS,NaN,...,NaN,NaN,NaN,1980.0,2004,NaN,NaN,45.41332,-93.26739,2004-003-253424110001
1,0.0,0.93,NaN,NaN,N,NaN,24457.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,1974.0,2004,55005.0,NaN,45.41354,-93.2701,2004-003-253424110002
2,0.0,8.75,NaN,NaN,N,NaN,24442.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,1969.0,2004,55005.0,NaN,45.41318,-93.27344,2004-003-253424120001
3,0.0,11.17,NaN,NaN,N,NaN,410.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,1989.0,2004,55005.0,NaN,45.41167,-93.27684,2004-003-253424210002
4,0.0,14.46,NaN,NaN,N,NaN,480.0,NaN,SAINT FRANCIS,BETHEL,...,NaN,NaN,NaN,1995.0,2004,55070.0,NaN,45.41169,-93.27849,2004-003-253424210003


#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

In [95]:
key_set=set()
row_num=0

In [98]:
for file in first_chunks:
    file_with_key = add_key(file)
    key_set = key_set.union(set(file_with_key.key))
    row_num=row_num+len(file_with_key)  
    print(len(set(file_with_key.key)))

490
489
498
498
500
500
500
500
500
500
500


In [97]:
print(row_num)
print(len(key_set))

5500
5475


9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.